Load and Prepare JobList


In [1]:
import pandas as pd
import pymysql

# Connect to database
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='admintushar15',
    database='printer_data_db'
)

# Load JobList - this is tiny, only 1,408 rows!
jobs_df = pd.read_sql("SELECT * FROM JobList", connection)
print(f"Loaded {len(jobs_df)} jobs")

# Look at what we have
print(jobs_df.head())

Loaded 1408 jobs
                         date                   id  \
0  2024-04-09 15:56:58.697000  CZPX1622X017XC78456   
1  2024-04-11 09:41:14.293000  CZPX1522X017XC78087   
2  2024-04-11 09:43:09.177000  CZPX4921X017XC67390   
3  2024-04-11 10:04:51.453000  CZPX1522X017XC78087   
4  2024-04-11 10:05:03.598000  CZPX1522X017XC78307   

                                              name     size  mTimestamp  
0  controller holder v1_0.25mm_PLA_MINI_3h0m.gcode  3515583  1712671042  
1                           sample-test-37-5.gcode    32672  1711372336  
2                           sample-test-37-6.gcode    32672  1711372347  
3                           sample-test-37-6.gcode    32672  1711372336  
4                           sample-test-37-6.gcode    32672  1711372383  


/var/folders/n3/lvjf2dhd2xbczw56rt1rzqx80000gn/T/ipykernel_22282/3870653998.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  jobs_df = pd.read_sql("SELECT * FROM JobList", connection)


Create Job Windows


In [2]:
# Sort jobs by printer and time
jobs_df = jobs_df.sort_values(['id', 'date'])

# For each job, find when the NEXT job starts
# This tells us when current job ends
jobs_df['next_job_start'] = jobs_df.groupby('id')['date'].shift(-1)

# The last job for each printer goes until "now"
jobs_df['next_job_start'] = jobs_df['next_job_start'].fillna('2025-12-31')

print("Job windows created:")
print(jobs_df[['id', 'date', 'next_job_start']].head(10))

Job windows created:
                     id                        date  \
1   CZPX1522X017XC78087  2024-04-11 09:41:14.293000   
3   CZPX1522X017XC78087  2024-04-11 10:04:51.453000   
6   CZPX1522X017XC78087  2024-04-11 10:37:05.060000   
7   CZPX1522X017XC78087  2024-04-11 10:39:45.890000   
15  CZPX1522X017XC78087  2024-04-11 11:01:01.684000   
22  CZPX1522X017XC78087  2024-04-11 13:57:22.413000   
25  CZPX1522X017XC78087  2024-04-16 11:06:00.496000   
30  CZPX1522X017XC78087  2024-04-16 12:14:54.157000   
36  CZPX1522X017XC78087  2024-04-16 13:19:32.015000   
39  CZPX1522X017XC78087  2024-04-16 13:34:59.909000   

                next_job_start  
1   2024-04-11 10:04:51.453000  
3   2024-04-11 10:37:05.060000  
6   2024-04-11 10:39:45.890000  
7   2024-04-11 11:01:01.684000  
15  2024-04-11 13:57:22.413000  
22  2024-04-16 11:06:00.496000  
25  2024-04-16 12:14:54.157000  
30  2024-04-16 13:19:32.015000  
36  2024-04-16 13:34:59.909000  
39  2024-04-16 23:22:13.683000  
